In [112]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network 
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
import random
from scipy.stats import beta
import pandas as pd
import copy
%matplotlib inline
#%run pure_strategy_selection.ipynb  #include simple selection algorithm
import scipy.io
import collections
import sys


## Mathmatic Functions

### Testing players' behavior

In [6]:

def MinMax_play(s,n):    # maxmizer first-time play, greedy algorithm
    # print('Minimizer Play')
   
    (v1, min_opinion, min_pol, max_action) = choose_min_vertex(s, n) # The best choice among all opinions and vertexs, function is in "pure_strategy_selection.ipynb"
    (v2, max_opinion) = max_action
    if v1 == None:   # if maximizer cannot find one
        print('Maximizer fail')

    else:
        # print("                                ")
        # print("Maximizer finds its target agent:")
#         print('v1', 'changed_opinion', 'innate_obj', 'obj')
#         print(max_champion)

        #Store innate_op of the max_selected vertex
        old_opinion_min = s[v1, 0]
        old_opinion_max = s[v2, 0]
        ##### change the agent's opinion with best action(agent v1, max_op)

        ## check if agent's opinionis is changed or not
        # print("Max Action:    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
        # print("Min Action:    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))
        # print("Network reaches equilibrium Polarization: " + str(min_pol))


    return(v1, v2, min_opinion, min_pol)

In [8]:
# determine which agent maximizer should select to maximizer the equilibrium polarization
def choose_min_vertex(s, n):
   # max_por = obj_polarization(A, L, op, n)  # use "innate"(after min action) polarization as a comparable standard to find max_por
    min_por = 10000
    C1 = list(range(n))    # for all agent 
    for v1 in C1:         
            # print('Minimizer start from agent'+str(v1))
            (min_opinion, por, max_action) = get_min_opinion(s, n, v1)
 
            if por < min_por: # if the polarization of most recent action > maximum polarization of previous actions
                min_por = por
                champion = (v1, min_opinion,min_por, max_action)   # save the this action as champion    
    # print("Min champion", champion)
    return (champion)


In [10]:
# determines if value of opinion at v should be set to 0 or 1 to maximize equilibrium polarization 
def get_min_opinion(s, n, v1):
    
    por_arr = np.zeros(11)  # create a two_element array to store polarization value of each option
    min_opi_option = np.arange(0, 1.01, 0.01) #Min has two options to change agent v2's opinion
    #min_opi_option = np.arange(0, 1.01, 0.1)
    #[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] 
    
    min_pol = 1000
    # objective if set opinion to 0, 1.0
    j = 0
    max_champion = (None, None)
    for min_opinion in min_opi_option:
        #print(v1,min_opinion)
        (v2,max_opinion, pol) = minmax_polar(s,v1,min_opinion)
        por_arr[j] = pol
        #print("max_pol: ", max_pol)
        j = j + 1   # index increase 1, put the polarization in array
        
        if  pol < min_pol:
            min_pol = pol
            max_champion = (v2,max_opinion)
    #print("Max_champion:",max_champion)   
    #print("Min choice:", por_arr)
    minmize_op = int(np.argmin(por_arr))/10  # the index of maximum polarization = max_opinion --[0,1]
    min_por = np.min(por_arr)        # find the maximum polarization in the record
    
    return (minmize_op, min_por, max_champion)

In [12]:
def minmax_polar(s, v1, min_opinion):
    op = copy.copy(s)
    op[v1] = min_opinion  
    (v2,max_opinion, max_pol) = maximizer_play(op,n,v1)
    
    return (v2,max_opinion, max_pol)
    
    

In [14]:
##### minimizer first-time play, greedy algorithm
def maximizer_play(op,n,min_touched): 
    
    op1 = copy.copy(op)
    max_champion = choose_max_vertex(op1, n, min_touched) 
    (v2, max_opinion, max_pol) = max_champion
    
    if v2 == None:
        print('Minimizer fail')

    return (v2,max_opinion, max_pol)

### Minimizer Strategy

In [17]:
# Minimizer search: Go through each agent 

def choose_max_vertex(op, n, min_touched):
    # current opinion array that changed by maximizer, "innate" opinion that min start with

    #champion = (None, None, 0, None)  # assume the best action is champion
    max_por = 0
    all = list(range(n))    # for all agent 
    C1 = [x for x in all if x != min_touched]  # for the vertice that Maximizer has not touched
    
    for v2 in C1:   
        #print('Max start with agent '+ str(v2) )
        (changed_opinion, por) = get_max_opinion(op,n,v2)   # find the best new_op option           

        if por > max_por:  # if the recent polarization is smaller than the minimum polarization in the history
            max_por = por                         # update the recent option as champion
            champion = (v2, changed_opinion, max_por)  
    # print("Max champion", champion)
    return (champion)  # find the best minimizer's action after going through every new_op option of every agent


In [19]:
# determines if value of opinion at v should be set to 0 or 1 to maximize equilibrium polarization 
def get_max_opinion(op, n, v2):
    
    por_arr = np.zeros(2)  # create a two_element array to store polarization value of each option
    max_opi_option = [0, 1.0]   # Maximizer has two options to change agent v1's opinion
    op1 = copy.copy(op)
    
    # objective if set opinion to 0, 1.0
    j = 0
    for max_opinion in max_opi_option:
        op1[v2] = max_opinion
#         print("max change opinion:",op1)
        max_pol = obj_polarization(A, op1, n)
        por_arr[j] = max_pol
        j = j + 1   # index increase 1, put the polarization in array

    maxmize_op = np.argmax(por_arr)  # the index of maximum polarization = max_opinion --[0,1]
    max_por = np.max(por_arr)        # find the maximum polarization in the record
    
    return (maxmize_op, max_por)